In [ ]:
## Import Modules 

import torch
import torchvision
from torch import nn, optim
import torch.nn.functional as F
from torchsummary import summary

In [ ]:
## Model Configuration (Initialize hyperparameters)
batch_size = 64
learning_rate = 0.01
cross_entropy = nn.CrossEntropyLoss()

In [ ]:
## DataLoader (Load the training set and validation set using Dataset and DataLoader)
transform = torchvision.transforms.ToTensor()
train_data = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(
    'mnist_data', train=True, download=True, transform=transform
    ), batch_size=batch_size
)
val_data = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(
    'mnist_data', train=False, download=True, transform=transform
    ), batch_size=batch_size
)

In [ ]:
# Validation function (To check whether the model is learning properly we can use a validation set)
def validate(model, data):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(data):
        images = images.cuda()
        labels = labels.cuda()
        y_pred = model(images)
        value, pred = torch.max(y_pred, 1)
        total += y_pred.size(0)
        correct += torch.sum(pred == labels)
    return correct * 100 / total

In [ ]:
## Training Function (Training the model)
def train(model,epochs=5) :
    optimizer = optim.Adam(model.parameters(),lr=learning_rate)    
    for n in range(epochs)  :
        for i , (images , labels) in enumerate(train_data) :
            images = images.cuda()
            labels = labels.cuda()
            optimizer.zero_grad()
            prediction = model(images)
            loss = cross_entropy(prediction, labels)
            loss.backward()
            optimizer.step()
        accuracy = float(validate(model, val_data))
        print("Epoch:", n+1, "Loss: ", float(loss.data), "Accuracy:", accuracy)

In [ ]:
## Model (A sample CNN is defined here for image classification)
class CNNWithPool(nn.Module) :
    def __init__(self):
        super(CNNWithPool,self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3)
        self.conv_2 = nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3)
        
        self.pool_1 = nn.MaxPool2d(2)
        self.pool_2 = nn.MaxPool2d(2)

        self.dense_1 = nn.Linear(in_features=800,out_features=256)
        self.dense_2 = nn.Linear(in_features=256,out_features=10)

        self.tanh = nn.Tanh()
    def forward(self,x) :
        x = self.tanh(self.conv_1(x))
        x = self.pool_1(x)
        x = self.tanh(self.conv_2(x))
        x = self.pool_2(x)
        x = x.view(x.shape[0],-1)
        x = self.tanh(self.dense_1(x))
        x = self.dense_2(x)
        # output = self.tanh(x)
        output = F.log_softmax(x, dim=1)

        return output

In [ ]:
# Model (Initialize the neural network)
model = CNNWithPool().cuda()

In [ ]:
# Summary
summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
              Tanh-2           [-1, 16, 26, 26]               0
         MaxPool2d-3           [-1, 16, 13, 13]               0
            Conv2d-4           [-1, 32, 11, 11]           4,640
              Tanh-5           [-1, 32, 11, 11]               0
         MaxPool2d-6             [-1, 32, 5, 5]               0
            Linear-7                  [-1, 256]         205,056
              Tanh-8                  [-1, 256]               0
            Linear-9                   [-1, 10]           2,570
Total params: 212,426
Trainable params: 212,426
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.25
Params size (MB): 0.81
Estimated Total Size (MB): 1.07
-------------------------------------------

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [ ]:
# Train for 30 Epochs
train(model,epochs=30)

Epoch: 1 Loss:  0.009247204288840294 Accuracy: 94.86000061035156
Epoch: 2 Loss:  0.0018443128792569041 Accuracy: 95.08999633789062
Epoch: 3 Loss:  0.2627520263195038 Accuracy: 95.13999938964844
Epoch: 4 Loss:  0.0019994506146758795 Accuracy: 96.39999389648438
Epoch: 5 Loss:  0.007705670315772295 Accuracy: 95.1199951171875
Epoch: 6 Loss:  0.04874978959560394 Accuracy: 94.05999755859375
Epoch: 7 Loss:  0.0525917112827301 Accuracy: 96.3499984741211
Epoch: 8 Loss:  0.029023030772805214 Accuracy: 95.39999389648438
Epoch: 9 Loss:  0.05686714127659798 Accuracy: 94.44999694824219
Epoch: 10 Loss:  0.006317119114100933 Accuracy: 94.80999755859375
Epoch: 11 Loss:  0.003050532890483737 Accuracy: 95.90999603271484
Epoch: 12 Loss:  0.004467764403671026 Accuracy: 96.08999633789062
Epoch: 13 Loss:  0.005192233249545097 Accuracy: 96.70999908447266
Epoch: 14 Loss:  0.006924818269908428 Accuracy: 97.00999450683594
Epoch: 15 Loss:  0.11356080323457718 Accuracy: 95.68000030517578
Epoch: 16 Loss:  0.0271630

**We can see that by using pooling, the number of parameters has been reduced, resulting in quicker training times without sacrificing the quality of the findings**